In [2]:
import torch

In [40]:
batch_size = 2
in_z = 2
in_h = 3
in_w = 4

scaling = 2
out_z = int(in_z * scaling)
out_h = int(in_h * scaling)
out_w = int(in_w * scaling)

input_tensor = torch.zeros((batch_size, 3, in_z, in_h, in_w))
# input_tensor[0, 0, :, :, :] = torch.linspace(0, 1.0, in_w)
# input_tensor[0, 1, :, :, :] = torch.linspace(0, 1.0, in_h)
# input_tensor[0, 2, :, :, :] = torch.linspace(0, 1.0, in_z)
for i in range(in_h):
    for j in range(in_w):
        for k in range(in_z):
            input_tensor[0, 0, k, i, j] = i / (in_h-1)
            input_tensor[0, 1, k, i, j] = j / (in_w-1)
            input_tensor[0, 2, k, i, j] = k / (in_z-1)

In [44]:
mesh_z, mesh_y, mesh_x = torch.meshgrid([
    torch.linspace(-1.0, 1.0, out_z),
    torch.linspace(-1.0, 1.0, out_h),
    torch.linspace(-1.0, 1.0, out_w),
])
grid = torch.stack((mesh_x, mesh_y, mesh_z), 3).tile((batch_size, 1, 1, 1, 1))
out = torch.nn.functional.grid_sample(
    input_tensor, 
    grid, 
    mode="bilinear", 
    align_corners=True
)
print(input_tensor.shape, grid.shape, out.shape)

torch.Size([2, 3, 2, 3, 4]) torch.Size([2, 4, 6, 8, 3]) torch.Size([2, 3, 4, 6, 8])


In [42]:
for c in range(input_tensor.shape[1]):
    print(f"---{c}---")
    for z in range(input_tensor.shape[2]):
        print(input_tensor[0, c, z, :, :])
    print("-------")

---0---
tensor([[0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.5000, 0.5000],
        [1.0000, 1.0000, 1.0000, 1.0000]])
tensor([[0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.5000, 0.5000],
        [1.0000, 1.0000, 1.0000, 1.0000]])
-------
---1---
tensor([[0.0000, 0.3333, 0.6667, 1.0000],
        [0.0000, 0.3333, 0.6667, 1.0000],
        [0.0000, 0.3333, 0.6667, 1.0000]])
tensor([[0.0000, 0.3333, 0.6667, 1.0000],
        [0.0000, 0.3333, 0.6667, 1.0000],
        [0.0000, 0.3333, 0.6667, 1.0000]])
-------
---2---
tensor([[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]])
tensor([[1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.]])
-------


In [43]:
for c in range(out.shape[1]):
    print(f"---{c}---")
    for z in range(out.shape[2]):
        print(out[0, c, z, :, :])
    print("-------")

---0---
tensor([[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.2000],
        [0.4000, 0.4000, 0.4000, 0.4000, 0.4000, 0.4000, 0.4000, 0.4000],
        [0.6000, 0.6000, 0.6000, 0.6000, 0.6000, 0.6000, 0.6000, 0.6000],
        [0.8000, 0.8000, 0.8000, 0.8000, 0.8000, 0.8000, 0.8000, 0.8000],
        [1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000]])
tensor([[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.2000],
        [0.4000, 0.4000, 0.4000, 0.4000, 0.4000, 0.4000, 0.4000, 0.4000],
        [0.6000, 0.6000, 0.6000, 0.6000, 0.6000, 0.6000, 0.6000, 0.6000],
        [0.8000, 0.8000, 0.8000, 0.8000, 0.8000, 0.8000, 0.8000, 0.8000],
        [1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000]])
tensor([[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2

In [45]:
from typing import *


def resize_3d_image(img: torch.Tensor, new_whd: Tuple[int, int, int]):
    out_w, out_h, out_d = new_whd
    batch_size, _c, in_d, in_h, in_w = img.shape
    mesh_z, mesh_y, mesh_x = torch.meshgrid([
        torch.linspace(-1.0, 1.0, out_d),
        torch.linspace(-1.0, 1.0, out_h),
        torch.linspace(-1.0, 1.0, out_w),
    ])
    grid = torch.stack((mesh_x, mesh_y, mesh_z), 3).tile((batch_size, 1, 1, 1, 1)).to(img.device)
    out = torch.nn.functional.grid_sample(
        img,
        grid,
        mode="bilinear",
        align_corners=True
    )
    return out

In [46]:
device = "cuda"
input_tensor = input_tensor.to(device)
out = resize_3d_image(input_tensor, (out_w, out_h, out_z))

In [47]:
for c in range(input_tensor.shape[1]):
    print(f"---{c}---")
    for z in range(input_tensor.shape[2]):
        print(input_tensor[0, c, z, :, :])
    print("-------")

---0---
tensor([[0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.5000, 0.5000],
        [1.0000, 1.0000, 1.0000, 1.0000]], device='cuda:0')
tensor([[0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.5000, 0.5000],
        [1.0000, 1.0000, 1.0000, 1.0000]], device='cuda:0')
-------
---1---
tensor([[0.0000, 0.3333, 0.6667, 1.0000],
        [0.0000, 0.3333, 0.6667, 1.0000],
        [0.0000, 0.3333, 0.6667, 1.0000]], device='cuda:0')
tensor([[0.0000, 0.3333, 0.6667, 1.0000],
        [0.0000, 0.3333, 0.6667, 1.0000],
        [0.0000, 0.3333, 0.6667, 1.0000]], device='cuda:0')
-------
---2---
tensor([[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]], device='cuda:0')
tensor([[1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.]], device='cuda:0')
-------


In [48]:
for c in range(out.shape[1]):
    print(f"---{c}---")
    for z in range(out.shape[2]):
        print(out[0, c, z, :, :])
    print("-------")

---0---
tensor([[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.2000],
        [0.4000, 0.4000, 0.4000, 0.4000, 0.4000, 0.4000, 0.4000, 0.4000],
        [0.6000, 0.6000, 0.6000, 0.6000, 0.6000, 0.6000, 0.6000, 0.6000],
        [0.8000, 0.8000, 0.8000, 0.8000, 0.8000, 0.8000, 0.8000, 0.8000],
        [1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000]],
       device='cuda:0')
tensor([[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.2000],
        [0.4000, 0.4000, 0.4000, 0.4000, 0.4000, 0.4000, 0.4000, 0.4000],
        [0.6000, 0.6000, 0.6000, 0.6000, 0.6000, 0.6000, 0.6000, 0.6000],
        [0.8000, 0.8000, 0.8000, 0.8000, 0.8000, 0.8000, 0.8000, 0.8000],
        [1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000]],
       device='cuda:0')
tensor([[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.000